In [ ]:
#numpy
import numpy as np

class Neural_Network(object):
    '''
    Create a 3 layered (input, hidden, output) feedforward neural network with back propagation and bias. 
    '''
    
    def __init__(self, input_width, hidden_width, learning_rate, number_of_epochs):
        '''
        Parameters:
        ----------
            input_width: (int) number of nodes in input layer
            hidden_width: (int) number of nodes in hidden layer
            learning_rate: (float) learning rate 
            number_of_epochs: (int) how many epochs to use when training
        '''
        
        # Widths of layers
        self.InputWidth = input_width
        self.HiddenWidth = hidden_width
        self.OutputWidth = 1
        
        # Weights
        self.Weights1 = np.random.randn(self.InputWidth,self.HiddenWidth)*0.2
        self.Weights2 = np.random.randn(self.HiddenWidth,self.OutputWidth)*0.2
        
        # Hidden Layer biases
        self.b1 = np.random.randn(self.HiddenWidth)
        self.b2 = np.random.randn(self.OutputWidth)
        
        # learning rate
        self.eta = learning_rate 
        
        # number of epochs
        self.epochs = number_of_epochs
        
   
    # Sigmoid activation function and derivative
    def Sigmoid(self,z):
        '''
        Parameters:
        ----------
            z: (float) number to plug into sigmoid function
        '''
        return 1./(1.+np.exp(-np.float128(z)))
    
    
    def DerivSigmoid(self,z):
        '''
        Parameters:
        ----------
            z: (float) number to plug into derivative of sigmoid function
            sigmoid already applied to z
        '''
        return self.Sigmoid(z)*(1.-self.Sigmoid(z))
#         return z*(1.-z)
    
    
    # Propagation functions
    def FeedForward(self,X):
        '''
        Parameters: 
        ----------
            X: dataset of inputs
        '''
        # feed through whole neural network
        self.First = np.dot(X, self.Weights1) + self.b1
        self.Second = self.Sigmoid(self.First)
        Third = np.dot(self.Second,self.Weights2) #+ self.b2
        
        return Third
        
    
    def BackPropagation(self,X,yPredicted,yObserved): 
        '''
        Parameters:
        ----------
            X: dataset of inputs
            yPredicted: known output for X
            yObserved: calculated outputs going from input layer to output layer once
        '''
        # Output layer error and output delta
        self.OutputError = yPredicted - yObserved
        self.OutputDelta = self.OutputError*self.DerivSigmoid(yObserved)
        
        # Hidden layer error and hidden delta
        self.HiddenError = np.dot(self.OutputDelta,self.Weights2.T)
        self.HiddenDelta = self.HiddenError*self.DerivSigmoid(self.First)
        #self.HiddenDelta = self.HiddenError*self.DerivSigmoid(self.Second)
        
        # Update weights 
        self.Weights1 += self.eta*np.dot(X.T,self.HiddenDelta)
        self.Weights2 += self.eta*np.dot(self.Second.T,self.OutputDelta)
        
#         print(self.b)
#         print(np.ones(self.HiddenWidth))
#         print(self.HiddenDelta)

#         print('bias')
#         print(self.b)
#         print('bias adjust')
#         print(self.HiddenWidth)
#         print(self.HiddenDelta)
#         print(np.shape(self.HiddenDelta))
#         print(self.eta*np.dot(np.ones(self.HiddenWidth).T,self.HiddenDelta))
#         print(np.add(self.b,self.eta*np.dot(np.ones(self.HiddenWidth),self.HiddenDelta)))
        
        # Updates biases
        self.b1 += self.eta*np.dot(np.ones(len(self.First)),self.HiddenDelta)
#         self.b2 += self.eta*np.dot(np.ones(len(self.FeedForward(X))),self.OutputDelta)
#         self.b = self.b + self.eta*np.dot(np.ones(self.HiddenWidth),self.HiddenDelta)
#         self.b = np.add(self.b,self.eta*np.dot(np.ones(self.HiddenWidth),self.HiddenDelta))
    
   
    # Train on dataset
    def Train(self,X,yPredicted):
        '''
        Train neural network on data in epochs and print final loss
        
        Parameters:
        ----------
            X: dataset of inputs to train
            yPredicted: known output for X
        '''
        
        # feedforward and backpropagate for the specified number of epochs
        for i in range(self.epochs):
            self.BackPropagation(X,yPredicted,self.FeedForward(X))
        
        # print out loss
        print('Training Loss (MSE): ', np.mean(np.square(yPredicted - self.FeedForward(X))))
        
    
    # Predict output from new input
    def Predict(self,XNew):
        '''
        Predict the output given a new dataset XNew
        '''
        return self.FeedForward(XNew)
    
    
#     # Utility functions
#     def SaveWeights(self):
#         '''
#         Save weights from the input layer to the hidden layer as 'Weights1.txt'
#         and save weights from hidden layer to output as 'Weights2.txt'.
#         '''
#         np.savetxt('Weights1.txt',self.Weights1,fmt='%s')
#         np.savetxt('Weights2.txt',self.Weights2,fmt='%s')
#         np.savetxt('b.txt',self.b1,fmt='%s')

In [4]:
# PARSEC Isochrones

#astropy
import astropy
from astropy.io import fits, ascii
from astropy.table import Table, Column, QTable

parsecall = ascii.read('/Users/joshpovick/Desktop/Research/StellAANN/parsec3_3.dat', \
                    format='basic', delimiter='\s')

rgb = np.where(parsecall['label']==3)
parsec = parsecall[rgb]

In [13]:
round(len(parsec)*.8)

11957

In [11]:
### Find all models
### Model generation
import itertools as it

#transform data to new basis
new_data = np.array([parsec['logTe'],parsec['MH'],parsec['logg']])

#create new variables

# linear terms
x1 = new_data[:,0]
x2 = new_data[:,1]
x3 = new_data[:,2]

# linear int
x12 = np.multiply(x1,x2)
x13 = np.multiply(x1,x3)
x23 = np.multiply(x2,x3)

# squares
x1sq = x1**2
x2sq = x2**2
x3sq = x3**2

# cubes
x1cu = x1**3
x2cu = x2**3
x3cu = x3**3


#find all possible models
models = []
models_str = []

all_var_str = ['x1','x2','x3']#,'x12','x13','x23','x1sq','x2sq','x3sq','x1cu','x2cu','x3cu']
all_var = [x1,x2,x3]#,x12,x13,x23,x1sq,x2sq,x3sq,x1cu,x2cu,x3cu]

for i in range(1,len(all_var)+1):
    for subset in it.combinations(all_var,i):
        models.append(subset)
    for subset_str in it.combinations(all_var_str,i):
        models_str.append(np.array(subset_str))
        
models = np.array(models)
models_str = np.array(models_str)

In [12]:
len(models_str)

7

In [ ]:
len(models)